In [ ]:
import pandas as pd
import numpy as np

import sys, os

os.chdir(sys.path[0])
os.chdir('../EEG')

from _parameters import *


In [ ]:
def process(sub):

    log_edit = pd.DataFrame()

    for ses in sessions:

        # Load session data
        ses_fname = f'/log_s{sub}{ses}.csv'
        log = pd.read_csv(dirs['log'] + ses_fname)

        log.insert(0, 'session', ses)
        log.insert(0, 'subject', sub)

        same = log['order'] == 'first'
        rev = log['order'] == 'second'
        vals = ['low', 'high', 'low']

        # Calc tradeoff
        log['DT_diff'] = log['DT1']-log['DT2']
        log['DT_trade'] = log['DT1']-log['DT2']
        log['err_diff'] = log['diff1']-log['diff2']
        log['err_trade'] = log['diff1']-log['diff2']

        # Median split trade-off
        for var in ['DT_trade', 'err_trade']:

            med_same = log[var][same].median()
            med_rev = log[var][rev].median()

            conds = [
                same & (log[var] < med_same),
                same & (log[var] >= med_same),
                rev & (log[var] < med_rev),
            ]

            log[var] = np.select(conds, vals, default='high')

        # Mark bad DT
            
        # Find bad DT
        thres_DT1 = log['DT1'].mean() + 2.5 * log['DT1'].std()
        thres_DT2 = log['DT2'].mean() + 2.5 * log['DT2'].std()

        log['bad_DT'] = (log['DT1'] >= thres_DT1) | (log['DT2'] >= thres_DT2)
        
        # Concat sessions
        log_edit = pd.concat([log_edit, log], axis = 0, ignore_index=True)

    # Save sub data
    log_fname = dirs['log'] + f'/log_s{sub}_edit.csv'
    log_edit.to_csv(log_fname, index=False)

    return log_edit

In [ ]:
# Process all and save

log_all = []

for sub in subjects:
    log_edit = process(sub)
    log_all.append(log_edit)

log_all = pd.concat(log_all, axis = 0, ignore_index=True)

log_fname = dirs['log'] + '/log_edit_all.csv'
log_all.to_csv(log_fname, index=False)